In [1]:
from PIL import Image
from numpy import array
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# for neural network layers
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [2]:
data = np.load("zdmdtImages.npz")
print data['Y'][:-3].shape,data['X'].shape

batch_size = 32
num_classes = 2
epochs = 12
# splitting data into train and test
x_train, x_test, y_train, y_test= train_test_split(data['X'], data['Y'][:-3], test_size=0.20, random_state=42, stratify=data['Y'][:-3])
# splitting train into valid dataset
print x_train.shape
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.20, random_state=42, stratify=y_train)
print x_train.shape
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(35425,) (35425, 22, 24, 4)
(28340, 22, 24, 4)
(22672, 22, 24, 4)
('x_train shape: ', (22672, 22, 24, 4))
(22672, 'train samples')
(7085, 'test samples')


In [3]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2),
                 activation = 'relu',
                 input_shape = (22, 24, 4)))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Dropout(0.1))
model.add(Conv2D(64,kernel_size=(5, 5), activation = 'relu'))
model.add(Conv2D(128,kernel_size=(5, 5), activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()
class_weight = {0: 1.,
                1: 1.}

#try mean squared error for loss, patience to 10, kernel size consistent to (3,3), categorical crossentropy loss
# and look at upsilon on github, increase epocs marginally, binary classification for other classes, 
# the number of color filters should be 3 instead of 4, try taking images directly as input to NN
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adagrad(),
              metrics=['accuracy'])
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-6,patience=10,verbose=0, mode='auto')

model.fit(x_train, y_train,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(x_valid,y_valid),
          callbacks=[earlyStopping],
         class_weight=class_weight)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 21, 23, 32)        544       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 12, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 11, 12, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 8, 64)          51264     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 4, 128)         204928    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               786944    
__________

In [6]:
ypred = model.predict(x_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1),ypred.argmax(axis=1))

array([[6015,  134],
       [ 581,  355]])

In [7]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test.argmax(axis=1), ypred.argmax(axis=1))

(array([ 0.91191631,  0.72597137]),
 array([ 0.97820784,  0.3792735 ]),
 array([ 0.94389957,  0.49824561]),
 array([6149,  936]))

In [12]:
# new attempt to deep network
from keras.layers.advanced_activations import LeakyReLU, PReLU

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation = 'relu',
                 input_shape = (22, 24, 4)))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Dropout(0.1))
model.add(Conv2D(64,kernel_size=(5, 5), activation = 'relu'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(128,kernel_size=(5, 5), activation = 'relu'))
model.add(LeakyReLU(alpha=0.1))
model.add(Flatten())
model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

#try mean squared error for loss, patience to 10, kernel size consistent to (3,3), categorical crossentropy loss
# and look at upsilon on github, increase epocs marginally, binary classification for 
# other classes, the number of color filters should be 3 instead of 4, try taking 
# images directly as input to NN
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adagrad(),
              metrics=['accuracy'])
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-6,patience=10,verbose=0, mode='auto')

model.fit(x_train, y_train,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(x_valid,y_valid),
          callbacks=[earlyStopping])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 20, 22, 32)        1184      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 11, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 11, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 6, 7, 64)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 2, 3, 128)         204928    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 2, 3, 128)         0         
__________

In [13]:
ypred = model.predict(x_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1),ypred.argmax(axis=1))

array([[6008,  141],
       [ 535,  401]])

In [8]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test.argmax(axis=1), ypred.argmax(axis=1))

(array([ 0.9256917 ,  0.61315789]),
 array([ 0.95218735,  0.49786325]),
 array([ 0.93875261,  0.5495283 ]),
 array([6149,  936]))

In [10]:
# To be tried ...
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

accuracy = x_train.history['acc']
val_accuracy = x_train.history['val_acc']
loss = x_train.history['loss']
val_loss = x_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'history'